In [1]:
import numpy as np
import tensorflow as tf
from collections import OrderedDict #순서까지 기억하는 사전형
import os, random
import pickle
# cifar 10 
#wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
#tar xzf cifar-10-python.tar.gz
# resnet
#http://laonple.blog.me/220761052425
#http://laonple.blog.me/220770760226

In [2]:
# 파라미터들
input_size = 32
data_path = 'cifar10/cifar-10-batches-py/'

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']


list_class=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
n_side = 32
n_size = n_side * n_side
n_channel = 3

n_class = len(list_class)
n_layer = 3 #simple resnet

n_batch = 100

#input & output 
x = tf.placeholder("float", [n_batch, n_size * n_channel]) #batch x image size
y = tf.placeholder("float", [n_batch, n_class]) #batch x class

#weight&bias
stddev = 0.1
weights = {
    'c': tf.Variable(tf.random_normal([3, 3, n_channel, 16], stddev=stddev)),
    'c1_1': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_2': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_3': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_4': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c2_1': tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=stddev)),
    'c2_2': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c2_3': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c2_4': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c3_1': tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=stddev)),
    'c3_2': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'c3_3': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'c3_4': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'd1' :tf.Variable(tf.random_normal([32*32*64, n_class], stddev=stddev)),
    #'d2' :tf.Variable(tf.random_normal([1000, n_class], stddev=stddev))
}
biases = {
    'c': tf.Variable(tf.random_normal([16], stddev=stddev)), 
    'c1_1': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_2': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_3': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_4': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c2_1': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_2': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_3': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_4': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c3_1': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_2': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_3': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_4': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'd1': tf.Variable(tf.random_normal([n_class], stddev=stddev)),
    #'d2': tf.Variable(tf.random_normal([n_class], stddev=stddev))
}

#시작은 맥스풀
#끝은 에버리지 풀 
# 3x3 conv 2개마다 2개 이전의 결과를 더해줌 
#마지막은 Fc1000

In [3]:
#model
def ResNet(img_width, img_height, img_channel, _x, _w, _b, scope='Simple_ResNet'):
    _names = OrderedDict()

    #reshape
    _x_r = tf.reshape(_x, shape=[-1, img_width, img_height, img_channel])
    
    with tf.variable_scope(scope):
        #max pool
        #_pool = tf.nn.max_pool(_x_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        #_names['pool'] = _pool
        
        #convolution
        _conv = tf.nn.conv2d(_x_r, _w['c'], strides=[1,1,1,1], padding='SAME')
        _conv = tf.nn.bias_add(_conv, _b['c'])
        _conv = tf.nn.relu(_conv)
        _names['conv'] = _conv
        
        #residual conv - 1
        conv1_1 = tf.nn.conv2d(_conv, _w['c1_1'], strides=[1,1,1,1], padding='SAME')
        conv1_1 = tf.nn.bias_add(conv1_1, _b['c1_1'])
        conv1_1 = tf.nn.relu(conv1_1)
        _names['conv1_1'] = conv1_1
        
        conv1_2 = tf.nn.conv2d(conv1_1, _w['c1_2'], strides=[1,1,1,1], padding='SAME')
        conv1_2 = tf.nn.bias_add(conv1_2, _b['c1_2'])
        conv1_2 = tf.add(conv1_2 , _conv) #residual? 
        conv1_2 = tf.nn.relu(conv1_2)
        _names['conv1_2'] = conv1_2
        
        conv1_3 = tf.nn.conv2d(conv1_2, _w['c1_3'], strides=[1,1,1,1], padding='SAME')
        conv1_3 = tf.nn.bias_add(conv1_3, _b['c1_3'])
        conv1_3 = tf.nn.relu(conv1_3)
        _names['conv1_3'] = conv1_3
        
        conv1_4 = tf.nn.conv2d(conv1_3, _w['c1_4'], strides=[1,1,1,1], padding='SAME')
        conv1_4 = tf.nn.bias_add(conv1_4, _b['c1_4'])
        conv1_4 = tf.add(conv1_4 , conv1_2) #residual? 
        conv1_4 = tf.nn.relu(conv1_4)
        _names['conv1_4'] = conv1_4
        
        #residual conv - 2
        conv2_1 = tf.nn.conv2d(conv1_4, _w['c2_1'], strides=[1,1,1,1], padding='SAME')
        conv2_1 = tf.nn.bias_add(conv2_1, _b['c2_1'])
        conv2_1 = tf.nn.relu(conv2_1)
        _names['conv2_1'] = conv2_1
        
        conv2_2 = tf.nn.conv2d(conv2_1, _w['c2_2'], strides=[1,1,1,1], padding='SAME')
        conv2_2 = tf.nn.bias_add(conv2_2, _b['c2_2'])
        #conv2_2 = tf.add(conv2_2, conv1_4)
        conv2_2 = tf.add(conv2_2, conv2_1)
        conv2_2 = tf.nn.relu(conv2_2)
        _names['conv2_2'] = conv2_2
        
        conv2_3 = tf.nn.conv2d(conv2_2, _w['c2_3'], strides=[1,1,1,1], padding='SAME')
        conv2_3 = tf.nn.bias_add(conv2_3, _b['c2_3'])
        conv2_3 = tf.nn.relu(conv2_3)
        _names['conv2_3'] = conv2_3
        
        conv2_4 = tf.nn.conv2d(conv2_3, _w['c2_4'], strides=[1,1,1,1], padding='SAME')
        conv2_4 = tf.nn.bias_add(conv2_4, _b['c2_4'])
        conv2_4 = tf.add(conv2_4, conv2_2)
        conv2_4 = tf.nn.relu(conv2_4)
        _names['conv2_4'] = conv2_4
        
        #residual conv - 3
        conv3_1 = tf.nn.conv2d(conv2_4, _w['c3_1'], strides=[1,1,1,1], padding='SAME')
        conv3_1 = tf.nn.bias_add(conv3_1, _b['c3_1'])
        conv3_1 = tf.nn.relu(conv3_1)
        _names['conv3_1'] = conv3_1
        
        conv3_2 = tf.nn.conv2d(conv3_1, _w['c3_2'], strides=[1,1,1,1], padding='SAME')
        conv3_2 = tf.nn.bias_add(conv3_2, _b['c3_2'])
        #conv3_2 = tf.add(conv3_2, conv2_4)
        conv3_2 = tf.add(conv3_2, conv3_1)
        conv3_2 = tf.nn.relu(conv3_2)
        _names['conv3_2'] = conv3_2
        
        conv3_3 = tf.nn.conv2d(conv3_2, _w['c3_3'], strides=[1,1,1,1], padding='SAME')
        conv3_3 = tf.nn.bias_add(conv3_3, _b['c3_3'])
        conv3_2 = tf.nn.relu(conv3_3)
        _names['conv3_3'] = conv3_3
        
        conv3_4 = tf.nn.conv2d(conv3_3, _w['c3_4'], strides=[1,1,1,1], padding='SAME')
        conv3_4 = tf.nn.bias_add(conv3_4, _b['c3_4'])
        conv3_4 = tf.add(conv3_4, conv3_2)
        conv3_4 = tf.nn.relu(conv3_4)
        _names['conv3_4'] = conv3_4
        
        
        
        #vectorize
        _dense = tf.reshape(conv3_4, [-1, _w['d1'].get_shape().as_list()[0]])
        _names['dense'] = _dense
        
        #dense
        _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
        _names['logit'] = _logit
                
    return _names        

In [4]:
#prediction
out = ResNet(n_side, n_side, n_channel, x, weights, biases, 'Simple_ResNet')
print(out)

OrderedDict([('conv', <tf.Tensor 'Simple_ResNet/Relu:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_1', <tf.Tensor 'Simple_ResNet/Relu_1:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_2', <tf.Tensor 'Simple_ResNet/Relu_2:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_3', <tf.Tensor 'Simple_ResNet/Relu_3:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_4', <tf.Tensor 'Simple_ResNet/Relu_4:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv2_1', <tf.Tensor 'Simple_ResNet/Relu_5:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv2_2', <tf.Tensor 'Simple_ResNet/Relu_6:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv2_3', <tf.Tensor 'Simple_ResNet/Relu_7:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv2_4', <tf.Tensor 'Simple_ResNet/Relu_8:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv3_1', <tf.Tensor 'Simple_ResNet/Relu_9:0' shape=(100, 32, 32, 64) dtype=float32>), ('conv3_2', <tf.Tensor 'Simple_ResNet/Relu_10:0' shape=(100, 32, 32, 64) dtype=float32>), ('conv3_3', 

In [5]:
#cost function & loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out['logit']))

In [6]:
#optimizer
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [7]:
#initializer
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


In [8]:
print(y)
print(out['dense'])
print(out)

Tensor("Placeholder_1:0", shape=(100, 10), dtype=float32)
Tensor("Simple_ResNet/Reshape:0", shape=(100, 65536), dtype=float32)
OrderedDict([('conv', <tf.Tensor 'Simple_ResNet/Relu:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_1', <tf.Tensor 'Simple_ResNet/Relu_1:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_2', <tf.Tensor 'Simple_ResNet/Relu_2:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_3', <tf.Tensor 'Simple_ResNet/Relu_3:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv1_4', <tf.Tensor 'Simple_ResNet/Relu_4:0' shape=(100, 32, 32, 16) dtype=float32>), ('conv2_1', <tf.Tensor 'Simple_ResNet/Relu_5:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv2_2', <tf.Tensor 'Simple_ResNet/Relu_6:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv2_3', <tf.Tensor 'Simple_ResNet/Relu_7:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv2_4', <tf.Tensor 'Simple_ResNet/Relu_8:0' shape=(100, 32, 32, 32) dtype=float32>), ('conv3_1', <tf.Tensor 'Simple_ResNet/Relu_9:0' shape=(100, 32, 

In [9]:
#save
savedir = "nets/cnn_cifar10_resnet/"
saver = tf.train.Saver(max_to_keep=3)
save_step = 4
if not os.path.exists(savedir):
 os.makedirs(savedir)
print ("SAVER READY")

SAVER READY


In [10]:
def load_npz(file):
 
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        return dict
    
## Make batch_iterator for npz data
def make_batch_index(_data, batch_size=128, allow_small_batch=True):
    num_images = len(_data[b'data'])
    start_idx = list(range(0, num_images, batch_size))
    end_idx = list(range(batch_size, num_images + 1, batch_size))
    if allow_small_batch and end_idx[-1] < num_images :
        start_idx.append(end_idx[-1])
        end_idx.append(num_images)
    return zip(start_idx, end_idx)

def one_hot_encode(x, numclass):
    return np.eye(numclass)[x]

In [11]:
training_epochs = 1000
batch_size = 100
disp_each = 10
disp_batch =100
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

In [12]:
load_testdata=load_npz(test_file[0])

In [13]:
#training
for epoch in range(training_epochs):
    
    sample_list_new = random.sample(train_file, len(train_file))
    nNumBatch = 0
    AvgBatchCost = 0
    
    for index, file_name in enumerate(sample_list_new):
        file_name = data_path+file_name
        load_data = load_npz(file_name)
        batch_index = make_batch_index(load_data, batch_size, True)
        
        for start, end in batch_index:
            
            batch_xs = load_data[b'data'][start:end]
            batch_ys = one_hot_encode(load_data[b'labels'][start:end], n_class)
            #print(batch_ys)
            #print(batch_xs, shape)
            feeds = {x: batch_xs, y: batch_ys}
            sess.run(optm, feed_dict = feeds)
            nNumBatch+=1
            tmp_cost, _ = sess.run([cost, optm], feed_dict = feeds)
            AvgBatchCost += tmp_cost
        
        if nNumBatch % disp_batch == 0:
                train_acc = sess.run(accr, feed_dict = feeds)
                print('\t[%d nNumBatch] train_cost = %g, acc = %g'%(nNumBatch, AvgBatchCost/nNumBatch, train_acc))
                
    if epoch % disp_each == 0:
        num_batch = 0
        test_acc = 0
        avg_acc = 0
        load_test_data = load_npz(test_file[0])
        testdatalabel = one_hot_encode(load_testdata[b'labels'], n_class)
        test_batch_index = make_batch_index(load_data, batch_size, True)
                
        for start, end in test_batch_index:
            num_batch += 1
            batch_xs = load_testdata[b'data'][start:end]
            batch_ys = one_hot_encode(load_testdata[b'labels'][start:end],n_class)
            test_acc = sess.run(accr, feed_dict={x:batch_xs,y:batch_ys})
            avg_acc+=test_acc
        print('[%d epoch]test acc=%g'%(epoch, avg_acc/num_batch))

	[100 nNumBatch] train_cost = 1056.34, acc = 0.14
	[200 nNumBatch] train_cost = 534.928, acc = 0.2
	[300 nNumBatch] train_cost = 358.227, acc = 0.11
	[400 nNumBatch] train_cost = 269.459, acc = 0.2
	[500 nNumBatch] train_cost = 216.072, acc = 0.19
[0 epoch]test acc=0.1704
	[100 nNumBatch] train_cost = 2.30962, acc = 0.22
	[200 nNumBatch] train_cost = 2.27428, acc = 0.17
	[300 nNumBatch] train_cost = 2.25138, acc = 0.13
	[400 nNumBatch] train_cost = 2.23171, acc = 0.27
	[500 nNumBatch] train_cost = 2.2152, acc = 0.28
	[100 nNumBatch] train_cost = 2.12907, acc = 0.21
	[200 nNumBatch] train_cost = 2.12191, acc = 0.28
	[300 nNumBatch] train_cost = 2.11425, acc = 0.23
	[400 nNumBatch] train_cost = 2.11081, acc = 0.16
	[500 nNumBatch] train_cost = 2.10607, acc = 0.2
	[100 nNumBatch] train_cost = 2.05823, acc = 0.26
	[200 nNumBatch] train_cost = 2.054, acc = 0.21
	[300 nNumBatch] train_cost = 2.0412, acc = 0.34
	[400 nNumBatch] train_cost = 2.03301, acc = 0.23
	[500 nNumBatch] train_cost = 2.

	[400 nNumBatch] train_cost = 1.2545, acc = 0.61
	[500 nNumBatch] train_cost = 1.25243, acc = 0.59
	[100 nNumBatch] train_cost = 1.24403, acc = 0.62
	[200 nNumBatch] train_cost = 1.24241, acc = 0.67
	[300 nNumBatch] train_cost = 1.23919, acc = 0.57
	[400 nNumBatch] train_cost = 1.23901, acc = 0.6
	[500 nNumBatch] train_cost = 1.24415, acc = 0.55
	[100 nNumBatch] train_cost = 1.17738, acc = 0.61
	[200 nNumBatch] train_cost = 1.22804, acc = 0.57
	[300 nNumBatch] train_cost = 1.24318, acc = 0.63
	[400 nNumBatch] train_cost = 1.23705, acc = 0.64
	[500 nNumBatch] train_cost = 1.23647, acc = 0.54
	[100 nNumBatch] train_cost = 1.23618, acc = 0.56
	[200 nNumBatch] train_cost = 1.24448, acc = 0.66
	[300 nNumBatch] train_cost = 1.24682, acc = 0.58
	[400 nNumBatch] train_cost = 1.24117, acc = 0.52
	[500 nNumBatch] train_cost = 1.23836, acc = 0.61
	[100 nNumBatch] train_cost = 1.20622, acc = 0.58
	[200 nNumBatch] train_cost = 1.23138, acc = 0.67
	[300 nNumBatch] train_cost = 1.22208, acc = 0.57
	[

	[200 nNumBatch] train_cost = 1.08692, acc = 0.66
	[300 nNumBatch] train_cost = 1.10202, acc = 0.62
	[400 nNumBatch] train_cost = 1.11222, acc = 0.68
	[500 nNumBatch] train_cost = 1.12584, acc = 0.58
	[100 nNumBatch] train_cost = 1.09501, acc = 0.68
	[200 nNumBatch] train_cost = 1.09728, acc = 0.68
	[300 nNumBatch] train_cost = 1.11772, acc = 0.73
	[400 nNumBatch] train_cost = 1.12189, acc = 0.64
	[500 nNumBatch] train_cost = 1.126, acc = 0.66
	[100 nNumBatch] train_cost = 1.05953, acc = 0.67
	[200 nNumBatch] train_cost = 1.06874, acc = 0.66
	[300 nNumBatch] train_cost = 1.08988, acc = 0.66
	[400 nNumBatch] train_cost = 1.09709, acc = 0.63
	[500 nNumBatch] train_cost = 1.11071, acc = 0.72
	[100 nNumBatch] train_cost = 1.08099, acc = 0.66
	[200 nNumBatch] train_cost = 1.09326, acc = 0.73
	[300 nNumBatch] train_cost = 1.10632, acc = 0.59
	[400 nNumBatch] train_cost = 1.11317, acc = 0.61
	[500 nNumBatch] train_cost = 1.11695, acc = 0.61
	[100 nNumBatch] train_cost = 1.0099, acc = 0.58
	[2

	[100 nNumBatch] train_cost = 0.984708, acc = 0.7
	[200 nNumBatch] train_cost = 0.995296, acc = 0.67
	[300 nNumBatch] train_cost = 1.01876, acc = 0.68
	[400 nNumBatch] train_cost = 1.02346, acc = 0.69
	[500 nNumBatch] train_cost = 1.03496, acc = 0.66
	[100 nNumBatch] train_cost = 1.03283, acc = 0.7
	[200 nNumBatch] train_cost = 1.0174, acc = 0.77
	[300 nNumBatch] train_cost = 1.03021, acc = 0.69
	[400 nNumBatch] train_cost = 1.03847, acc = 0.62
	[500 nNumBatch] train_cost = 1.03719, acc = 0.7
	[100 nNumBatch] train_cost = 0.945581, acc = 0.76
	[200 nNumBatch] train_cost = 0.990411, acc = 0.67
	[300 nNumBatch] train_cost = 1.01812, acc = 0.71
	[400 nNumBatch] train_cost = 1.02457, acc = 0.67
	[500 nNumBatch] train_cost = 1.03346, acc = 0.64
[100 epoch]test acc=0.5269
	[100 nNumBatch] train_cost = 1.04282, acc = 0.67
	[200 nNumBatch] train_cost = 1.02756, acc = 0.69
	[300 nNumBatch] train_cost = 1.03771, acc = 0.72
	[400 nNumBatch] train_cost = 1.04873, acc = 0.67
	[500 nNumBatch] train_

	[300 nNumBatch] train_cost = 0.962166, acc = 0.72
	[400 nNumBatch] train_cost = 0.983841, acc = 0.73
	[500 nNumBatch] train_cost = 0.994093, acc = 0.62
[130 epoch]test acc=0.5361
	[100 nNumBatch] train_cost = 0.984096, acc = 0.71
	[200 nNumBatch] train_cost = 0.976659, acc = 0.65
	[300 nNumBatch] train_cost = 0.98172, acc = 0.68
	[400 nNumBatch] train_cost = 0.985827, acc = 0.65
	[500 nNumBatch] train_cost = 0.990443, acc = 0.67
	[100 nNumBatch] train_cost = 1.01244, acc = 0.71
	[200 nNumBatch] train_cost = 0.990718, acc = 0.65
	[300 nNumBatch] train_cost = 0.989918, acc = 0.7
	[400 nNumBatch] train_cost = 0.987859, acc = 0.74
	[500 nNumBatch] train_cost = 0.997117, acc = 0.68
	[100 nNumBatch] train_cost = 0.938339, acc = 0.75
	[200 nNumBatch] train_cost = 0.989347, acc = 0.69
	[300 nNumBatch] train_cost = 0.99102, acc = 0.72
	[400 nNumBatch] train_cost = 0.994629, acc = 0.66
	[500 nNumBatch] train_cost = 0.999429, acc = 0.6
	[100 nNumBatch] train_cost = 0.895553, acc = 0.74
	[200 nNu

...

	[300 nNumBatch] train_cost = 0.622217, acc = 0.85
	[400 nNumBatch] train_cost = 0.655546, acc = 0.4
	[500 nNumBatch] train_cost = 0.781524, acc = 0.69
	[100 nNumBatch] train_cost = 0.740139, acc = 0.79
	[200 nNumBatch] train_cost = 0.706846, acc = 0.81
	[300 nNumBatch] train_cost = 0.681362, acc = 0.82
	[400 nNumBatch] train_cost = 0.683982, acc = 0.73
	[500 nNumBatch] train_cost = 0.68149, acc = 0.72
	[100 nNumBatch] train_cost = 0.609578, acc = 0.83
	[200 nNumBatch] train_cost = 0.614036, acc = 0.81
	[300 nNumBatch] train_cost = 0.613572, acc = 0.84
	[400 nNumBatch] train_cost = 0.625212, acc = 0.85
	[500 nNumBatch] train_cost = 0.629158, acc = 0.82
	[100 nNumBatch] train_cost = 0.499044, acc = 0.9
	[200 nNumBatch] train_cost = 0.541255, acc = 0.87
	[300 nNumBatch] train_cost = 0.589887, acc = 0.81
	[400 nNumBatch] train_cost = 0.655962, acc = 0.75
	[500 nNumBatch] train_cost = 0.678361, acc = 0.76
	[100 nNumBatch] train_cost = 0.58943, acc = 0.89
	[200 nNumBatch] train_cost = 0.593

	[300 nNumBatch] train_cost = 0.639459, acc = 0.85
	[400 nNumBatch] train_cost = 0.626307, acc = 0.84
	[500 nNumBatch] train_cost = 0.625526, acc = 0.81
	[100 nNumBatch] train_cost = 0.515906, acc = 0.88
	[200 nNumBatch] train_cost = 0.556821, acc = 0.85
	[300 nNumBatch] train_cost = 0.591395, acc = 0.86
	[400 nNumBatch] train_cost = 0.60968, acc = 0.86
	[500 nNumBatch] train_cost = 0.626468, acc = 0.76
	[100 nNumBatch] train_cost = 0.713939, acc = 0.8
	[200 nNumBatch] train_cost = 0.676283, acc = 0.85
	[300 nNumBatch] train_cost = 0.665638, acc = 0.83
	[400 nNumBatch] train_cost = 0.662168, acc = 0.85
	[500 nNumBatch] train_cost = 0.666316, acc = 0.73
[900 epoch]test acc=0.4993
	[100 nNumBatch] train_cost = 0.688935, acc = 0.81
	[200 nNumBatch] train_cost = 0.666565, acc = 0.81
	[300 nNumBatch] train_cost = 0.636366, acc = 0.87
	[400 nNumBatch] train_cost = 0.632009, acc = 0.87
	[500 nNumBatch] train_cost = 0.637962, acc = 0.81
	[100 nNumBatch] train_cost = 0.590952, acc = 0.86
	[200 

	[300 nNumBatch] train_cost = 0.599886, acc = 0.83
	[400 nNumBatch] train_cost = 0.613121, acc = 0.8
	[500 nNumBatch] train_cost = 0.63338, acc = 0.81
[930 epoch]test acc=0.5105
	[100 nNumBatch] train_cost = 0.577115, acc = 0.79
	[200 nNumBatch] train_cost = 0.585114, acc = 0.86
	[300 nNumBatch] train_cost = 0.587479, acc = 0.8
	[400 nNumBatch] train_cost = 0.611684, acc = 0.93
	[500 nNumBatch] train_cost = 0.625284, acc = 0.82
	[100 nNumBatch] train_cost = 0.660641, acc = 0.82
	[200 nNumBatch] train_cost = 0.634618, acc = 0.75
	[300 nNumBatch] train_cost = 0.627583, acc = 0.85
	[400 nNumBatch] train_cost = 0.628695, acc = 0.89
	[500 nNumBatch] train_cost = 0.626579, acc = 0.89
	[100 nNumBatch] train_cost = 0.504679, acc = 0.88
	[200 nNumBatch] train_cost = 0.536903, acc = 0.9
	[300 nNumBatch] train_cost = 0.560221, acc = 0.87
	[400 nNumBatch] train_cost = 0.581043, acc = 0.81
	[500 nNumBatch] train_cost = 0.606133, acc = 0.79
	[100 nNumBatch] train_cost = 0.591637, acc = 0.86
	[200 nN

	[300 nNumBatch] train_cost = 0.644477, acc = 0.87
	[400 nNumBatch] train_cost = 0.644532, acc = 0.79
	[500 nNumBatch] train_cost = 0.639335, acc = 0.82
	[100 nNumBatch] train_cost = 0.615085, acc = 0.87
	[200 nNumBatch] train_cost = 0.706738, acc = 0.43
	[300 nNumBatch] train_cost = 0.968533, acc = 0.57
	[400 nNumBatch] train_cost = 0.989664, acc = 0.73
	[500 nNumBatch] train_cost = 0.975884, acc = 0.7
	[100 nNumBatch] train_cost = 0.711242, acc = 0.72
	[200 nNumBatch] train_cost = 0.919183, acc = 0.67
	[300 nNumBatch] train_cost = 0.903756, acc = 0.74
	[400 nNumBatch] train_cost = 0.87605, acc = 0.78
	[500 nNumBatch] train_cost = 0.861627, acc = 0.82
	[100 nNumBatch] train_cost = 0.700041, acc = 0.82
	[200 nNumBatch] train_cost = 0.918483, acc = 0.65
	[300 nNumBatch] train_cost = 0.862832, acc = 0.8
	[400 nNumBatch] train_cost = 0.837734, acc = 0.78
	[500 nNumBatch] train_cost = 0.809209, acc = 0.85
	[100 nNumBatch] train_cost = 0.678041, acc = 0.79
	[200 nNumBatch] train_cost = 0.65

	[300 nNumBatch] train_cost = 0.647363, acc = 0.74
	[400 nNumBatch] train_cost = 0.685934, acc = 0.81
	[500 nNumBatch] train_cost = 0.693776, acc = 0.83
	[100 nNumBatch] train_cost = 0.591132, acc = 0.87
	[200 nNumBatch] train_cost = 0.628018, acc = 0.81
	[300 nNumBatch] train_cost = 0.660909, acc = 0.84
	[400 nNumBatch] train_cost = 0.650235, acc = 0.88
	[500 nNumBatch] train_cost = 0.649945, acc = 0.85
	[100 nNumBatch] train_cost = 0.64052, acc = 0.81
	[200 nNumBatch] train_cost = 0.621547, acc = 0.92
	[300 nNumBatch] train_cost = 0.633359, acc = 0.76
	[400 nNumBatch] train_cost = 0.630591, acc = 0.82
	[500 nNumBatch] train_cost = 0.622666, acc = 0.82
	[100 nNumBatch] train_cost = 1.48369, acc = 0.47
	[200 nNumBatch] train_cost = 1.48509, acc = 0.52
	[300 nNumBatch] train_cost = 1.37072, acc = 0.72
	[400 nNumBatch] train_cost = 1.23572, acc = 0.77
	[500 nNumBatch] train_cost = 1.14274, acc = 0.81
	[100 nNumBatch] train_cost = 0.690173, acc = 0.88
	[200 nNumBatch] train_cost = 0.65466